There are 4 types of energy to predict <br>
    - 0 : electricity
    - 1 : chilledwater
    - 2 : steam
    - 3 : hotwater

Electricity and water consumption may have different behavior!<br>
     - I will make separately train & predict the model

* Reference 
 - https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type
 - https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction



In [1]:
# No leak data
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
%%time
# 파일 읽어오기
train_df = pd.read_csv('train.csv')
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_train_df = pd.read_csv('weather_train.csv')


test_df = pd.read_csv('test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

weather_test_df = pd.read_csv('weather_test.csv')
building_meta_df = pd.read_csv('building_metadata.csv')
sample_submission = pd.read_csv('sample_submission.csv')

Wall time: 59.7 s


In [3]:
# Glimpse of Data
print(train_df.shape)
print(weather_train_df.shape)
print(weather_test_df.shape)
print(building_meta_df.shape)

print(test_df.shape)

(20216100, 4)
(139773, 9)
(277243, 9)
(1449, 6)
(41697600, 4)


In [4]:
# ㅒ

In [5]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
# Reducing memory
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [7]:
def plot_date_usage(train_df, meter=0, building_id=0):
    train_temp_df = train_df[train_df['meter'] == meter]
    train_temp_df = train_temp_df[train_temp_df['building_id'] == building_id]    
    train_temp_df_meter = train_temp_df.groupby('date')['meter_reading_log1p'].sum()
    train_temp_df_meter = train_temp_df_meter.to_frame().reset_index()
    fig = px.line(train_temp_df_meter, x='date', y='meter_reading_log1p')
    fig.show()

## Feature engineering 
  - There are 3 parts to make features <br>
      train_df / weather_train_df / building_meta_df
  - and then I will merge them 

In [8]:
# train_df -- timestamp : 월 , 주, 일

train_df['meter_reading_log1p'] = np.log1p(train_df['meter_reading'])
# train_df['meter_reading']
# date / 월 / 주 /일 
train_df['date'] = train_df['timestamp'].dt.date
train_df['hour'] = train_df['timestamp'].dt.hour
train_df['weekend'] = train_df['timestamp'].dt.weekday
train_df['month'] = train_df['timestamp'].dt.month
train_df['dayofweek'] = train_df['timestamp'].dt.dayofweek


test_df['date'] = test_df['timestamp'].dt.date
test_df['hour'] = test_df['timestamp'].dt.hour
test_df['weekend'] = test_df['timestamp'].dt.weekday
test_df['month'] = test_df['timestamp'].dt.month
test_df['dayofweek'] = test_df['timestamp'].dt.dayofweek



In [9]:
# isholiday
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                "2017-01-01", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                "2019-01-01"]

train_df["is_holiday"] = (train_df.timestamp.dt.date.astype("str").isin(holidays)).astype(int)
test_df["is_holiday"] = (test_df.timestamp.dt.date.astype("str").isin(holidays)).astype(int)

In [10]:
#Outlier 제거 
# Remove outliers
train_df = train_df [ train_df['building_id'] != 1099 ]
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [11]:
# # summer / winter
# # summer = chilledwater 사용량이 많은 달. 

# train_df.head()
# # meter ==0 이 모두 0인 building id
# tmp = train_df[train_df['meter']==1][train_df['meter_reading']>0].groupby(['building_id','month'])['meter_reading'].count().reset_index()
# tmp[tmp['building_id']>1020][tmp['building_id']<1030]


In [12]:

weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [13]:
# weather - 
# weather data has a lot of nulls 
# I tried to fill these values by interpolating data
# df.groupby('').apply(lambda group: group.interpolate~~)

weather_train_df.head()
weather_train_df = weather_train_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))
weather_test_df = weather_test_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))

In [14]:

weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        17228
dew_temperature           0
precip_depth_1_hr     26273
sea_level_pressure     8755
wind_direction            0
wind_speed                0
dtype: int64

In [15]:
# lags 
# site 별로 최근 3일간의 날씨를 rolling 하기
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [16]:
add_lag_feature(weather_train_df, window=3)
add_lag_feature(weather_train_df, window=72)
add_lag_feature(weather_test_df, window=3)
add_lag_feature(weather_test_df, window=72)

In [17]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,air_temperature_mean_lag3,...,sea_level_pressure_min_lag72,sea_level_pressure_std_lag72,wind_direction_mean_lag72,wind_direction_max_lag72,wind_direction_min_lag72,wind_direction_std_lag72,wind_speed_mean_lag72,wind_speed_max_lag72,wind_speed_min_lag72,wind_speed_std_lag72
0,0,2016-01-01 00:00:00,25.000000,6.0,20.00000,-1.0,1019.5,0.0,0.000000,25.000000,...,1019.5,NaN,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.0,NaN
1,0,2016-01-01 01:00:00,24.406250,4.0,21.09375,-1.0,1020.0,70.0,1.500000,24.703125,...,1019.5,0.353516,35.000000,70.0,0.0,49.50000,0.750000,1.500000,0.0,1.060547
2,0,2016-01-01 02:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000,24.062500,...,1019.5,0.288574,23.328125,70.0,0.0,40.40625,0.500000,1.500000,0.0,0.866211
3,0,2016-01-01 03:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000,22.765625,...,1019.5,0.250000,17.500000,70.0,0.0,35.00000,0.375000,1.500000,0.0,0.750000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609,21.296875,...,1019.5,0.223633,64.000000,250.0,0.0,108.31250,0.819824,2.599609,0.0,1.188477


In [18]:
# # meter reading 값에 대한 aggregation
# # 과적합 문제를 야기할 수 있다.
# # meter 별로

# train_df['key'] =  train_df['building_id'].astype(str) + '__' + train_df['meter'].astype(str)

# df_group = train_df.groupby('key')['meter_reading_log1p']
# building_mean = df_group.mean().astype(np.float16)
# building_median = df_group.median().astype(np.float16)
# building_min = df_group.min().astype(np.float16)
# building_max = df_group.max().astype(np.float16)
# building_std = df_group.std().astype(np.float16)

# train_df['building_meter_mean'] = train_df['key'].map(building_mean)
# train_df['building_meter_median'] = train_df['key'].map(building_median)
# train_df['building_meter_min'] = train_df['key'].map(building_min)
# train_df['building_meter_max'] = train_df['key'].map(building_max)
# train_df['building_meter_std'] = train_df['key'].map(building_std)


# test_df['building_meter_mean'] = train_df['key'].map(building_mean)
# test_df['building_meter_median'] = train_df['key'].map(building_median)
# test_df['building_meter_min'] = train_df['key'].map(building_min)
# test_df['building_meter_max'] = train_df['key'].map(building_max)
# test_df['building_meter_std'] = train_df['key'].map(building_std)
# train_df = train_df.drop('key',axis =1)

In [19]:
# Removing weired data on site_id =0  
#https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113054#656588
# building_meta_df[building_meta_df.site_id == 0]

In [20]:
train_df.head()

,building_id,meter,timestamp,meter_reading,meter_reading_log1p,date,hour,weekend,month,dayofweek,is_holiday
103,105,0,2016-01-01,23.303600,3.190624,2016-01-01,0,4,1,4,1
104,106,0,2016-01-01,0.374600,0.318163,2016-01-01,0,4,1,4,1
105,106,3,2016-01-01,0.000000,0.000000,2016-01-01,0,4,1,4,1
106,107,0,2016-01-01,175.184006,5.171529,2016-01-01,0,4,1,4,1
107,108,0,2016-01-01,91.265297,4.524668,2016-01-01,0,4,1,4,1


### Merge


In [21]:
# 하나에 합치기

# base + building_meta_df 합치기
train_df = pd.merge(train_df,building_meta_df, on= ['building_id'],how='left')
test_df = pd.merge(test_df,building_meta_df, on= ['building_id'],how='left')
# del building_meta_df

In [22]:
# base + weather_train_df 합치기
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

train_df = pd.merge(train_df,weather_train_df, on= ['site_id','timestamp'],how='left')
test_df = pd.merge(test_df,weather_test_df, on= ['site_id','timestamp'],how='left')
# del weather_train_df, weather_test_df

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import train_test_split

### Encoding



In [24]:
#Encoidng variables
le = LabelEncoder()
# train_df['primary_use'] = train_df['primary_use'].astype(str)
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

# test_df['primary_use'] = test_df['primary_use'].astype(str)
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype(np.int8)

In [25]:
# Pickle 저장

train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')
del train_df, test_df
gc.collect()



14

In [2]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [3]:
# some feature enginnering

train_df['age'] = train_df['year_built'].max()-train_df['year_built']+1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

In [4]:
#Handling missing values
# To streamline this though process it is useful to know the 3 categories in which missing data can be classified into:

# Missing Completely at Random (MCAR)
# Missing at Random (MAR)
# Missing Not at Random (MNAR)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 


In [5]:
# drop_cols = ['date',"precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed","timestamp"]
drop_cols = ['date',"timestamp"]

target = train_df["meter_reading_log1p"]
del train_df["meter_reading"], train_df['meter_reading_log1p']
train_df = train_df.drop(drop_cols, axis=1)
drop_cols += ["row_id"]
# drop_cols.remove('date')
test_df = test_df.drop(drop_cols, axis=1)

In [6]:
train_df.head()
categorical_features = ["building_id", "site_id", "meter", "hour", "weekend",'month','is_holiday','primary_use']

In [7]:
folds = 3
seed = 99 #666

import lightgbm as lgb
from sklearn.model_selection import KFold
import xgboost as xgb

# lightbgm
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': {'l2'},
# #     'subsample': 0.2,
#     'learning_rate': 0.08,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.9,
# #     'alpha': 0.1,
# #     'lambda': 0.1,
# #     'n_jobs' :2 
# }

# params = {
#     "objective": "regression",
#     "boosting": "gbdt",
#     "num_leaves": 1280,
#     "learning_rate": 0.05,
#     "feature_fraction": 0.85,
#     "reg_lambda": 2,
#     "metric": "rmse",
#     "num_threads" : 10,
#     'seed' : seed
    
# }

#  n_estimators=10000,
#         max_depth=-1,
#         learning_rate=0.048,
#         subsample=0.85,
#         colsample_bytree=0.85,
#         missing=-999,
#         tree_method='gpu_hist',  # THE MAGICAL PARAMETER
#         reg_alpha=0.15,
#         reg_lamdba=0.85
#     )
    
params = {
    'colsample_bytree': 0.8,                 
    'learning_rate': 0.05,
    'max_depth': 10,
    'subsample': 0.8,
    'reg_alpha' :0.15,
    'reg_lamdba' : 0.85,
    'tree_method': 'gpu_hist',
    'missing' : -999,    
    'objective': 'reg:squarederror',
#     'n_jobs' : 4
}

In [8]:
# from sklearn.metrics import mean_squared_log_error
# from multiprocessing import Process as proc
# def xgbfitting(n):
#     print(2)
#     gbm = xgb.train(params, xgb_train, num_boost_round = 3000,evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
#                     verbose_eval = 30 , early_stopping_rounds = 30)
#     print(1)
#     gbm.save_model('xgb.model')
# #     model.save_model('tmp/xgb.model')

In [8]:
%%time
import pickle
# shuffle = False
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)
scores = [] 
models = []
for i,(train_index, val_index) in enumerate(kf.split(train_df)):
    train_X = train_df.iloc[train_index]
    val_X = train_df.iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
#     lgb_train = lgb.Dataset(train_X, train_y,categorical_feature=categorical_features)
#     lgb_eval = lgb.Dataset(val_X, val_y,categorical_feature=categorical_features)
    xgb_train = xgb.DMatrix(train_X, train_y)
    xgb_eval = xgb.DMatrix(val_X, val_y)
#     gbm = lgb.train(params,
#                     lgb_train,
#                     num_boost_round=1000, #300,
#                     valid_sets=(lgb_train, lgb_eval),
# #                     feval=rmsle,
#                     early_stopping_rounds= 50,#100,
#                     verbose_eval=30) #100)
    gbm = xgb.train(params, xgb_train, num_boost_round = 3000,evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
                    verbose_eval = 30 , early_stopping_rounds = 30)
#     print(11)
     
#     fitting_process = proc( target = xgbfitting, args = (i,) )
#     fitting_process.start()
#     fitting_process.join()
#     gbm.save_model('xgb_%s.model'%i)
    # save
    pickle.dump(gbm,open('xgb_%s.pickle'%i, "wb") )
    gbm.__del__() 
    print('delete')
#     print(gbm)
    del xgb_train, xgb_eval
    gc.collect()
    # CV Score 만들기
#     scores.append(gbm.predict(val_X))
    #MSLE
#scores.append( np.sqrt(mean_squared_log_error( np.expm1(val_y), np.expm1(predictions) )))
    
#     models.append(gbm)
#     del gbm
for i in range(folds):
    models.append(pickle.load(open("xgb_%s.pickle"%i, "rb")))²

C:\Users\yseon\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-rmse:4.09277	val-rmse:4.03497
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 30 rounds.
[30]	train-rmse:1.54774	val-rmse:1.67393
[60]	train-rmse:1.12906	val-rmse:1.33184
[90]	train-rmse:1.00773	val-rmse:1.24555
[120]	train-rmse:0.939901	val-rmse:1.20075
[150]	train-rmse:0.905979	val-rmse:1.17893
[180]	train-rmse:0.877122	val-rmse:1.1644
[210]	train-rmse:0.855618	val-rmse:1.15625
[240]	train-rmse:0.831678	val-rmse:1.14511
[270]	train-rmse:0.815483	val-rmse:1.13824
[300]	train-rmse:0.79704	val-rmse:1.13003
[330]	train-rmse:0.780804	val-rmse:1.12251
[360]	train-rmse:0.764842	val-rmse:1.11536
[390]	train-rmse:0.748067	val-rmse:1.10655
[420]	train-rmse:0.732962	val-rmse:1.10101
[450]	train-rmse:0.72119	val-rmse:1.09651
[480]	train-rmse:0.711523	val-rmse:1.09302
[510]	train-rmse:0.702501	val-rmse:1.08997
[540]	train-rmse:0.693576	val-rmse:1.08744
[570]	train-rmse:0.685494	val-rmse:1.08561
[600]	train

In [9]:
# %%time
# import matplotlib.pyplot as plt
# import seaborn as sns

# gbm = models[-1]
# feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(), gbm.feature_name()),reverse = True), columns=['Value','Feature'])
# plt.figure(figsize=(10, 15))

# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# # plt.title('LightGBM FEATURES')
# plt.tight_layout()
# plt.show()

In [10]:
# gbm.get_score(importance_type='gain')#,columns = ['Feature','Value'])


In [11]:
%%time
i = 0
res = []
res2 = []
res3 = []
step_size = 50000
for j in tqdm(range(int(np.ceil(test_df.shape[0] / 50000)))):
#     # 평균을 내고 나서 exp를 취하는 코드
#     res.append(np.expm1(sum([model.predict(test_df.iloc[i:i + step_size]) for model in models]) / folds))
    # exp를 취하고 평균을 내는 코드 https://www.kaggle.com/rohanrao/ashrae-half-and-half
    res2.append(sum([np.expm1(model.predict(xgb.DMatrix(test_df.iloc[i:i + step_size]))) for model in models])/ folds)
#     res3.append([np.expm1(model.predict(test_df.iloc[i:i + step_size])) for model in models])
    
    i += step_size

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [06:29<00:00,  2.46it/s]


Wall time: 6min 29s


In [12]:
%%time
from datetime import datetime
sample_submission = pd.read_csv('sample_submission.csv')
res2 = np.concatenate(res2)
sample_submission["meter_reading"] = res2
sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0
sample_submission.to_csv('xgb_exp_avg_sub_' + str(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')) + '.csv', index=False)
sample_submission.head(10)

Wall time: 1min 43s


,row_id,meter_reading
0,0,97.115349
1,1,44.861187
2,2,3.534665
3,3,172.153442
4,4,1048.248291
5,5,6.428292
6,6,50.771851
7,7,579.505066
8,8,272.677948
9,9,241.016922


In [ ]:
# %%time
# # 최근 데이터에 가중치를 더 주는 코드 
# res4 = []
# for i in range(len(res3)):
#     res4.append((0.6*res3[i][0]+0.3*res3[i][1]+0.1*res3[i][2]))

# res4 = np.concatenate(res4)
# sample_submission["meter_reading"] = res4
# sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0
# sample_submission.to_csv('exp_avg_recently_sub_' + str(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')) + '.csv', index=False)
# sample_submission.head(10)